# Data Parsing 

## The following cells will be debugging and development for a data parsing script utilizing pandas and system commands to access, create, and organize files. 

In [7]:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import datetime
import glob
import sys
import os
import time



In [8]:
custom_date_parser = lambda x,y,z,a,b: datetime.datetime.strptime(f"{x} {y} {z} {a} {b}", "%Y %m %d %H %M")


In [9]:
data_dir = f"..\\Data\\"


In [12]:
# Create the DataFrame that will be utilized to store all of the parsed data.
BigDF = pd.DataFrame()
# Store all of the files as a list, in order to access it in the loop appropriately.
files = glob.glob(f"../NREL Data/**/*{2020}.csv", recursive=True)
def dateparse(x): return pd.datetime.strptime(x, '%Y %m %d %H %M')

In [ ]:
for index in range(0, len(files)):
    start = time.time()
    if len(files) > 2:
        DataFile = files.pop()
        DataFile2 = files.pop()
        Csv_DF = pd.read_csv(DataFile, header=[2], parse_dates={
            'Date': [0, 1, 2, 3, 4]}, date_parser=dateparse)
        Lat_Lon_DF1 = pd.read_csv(DataFile, nrows=1)
        Csv_DF["Longitude"] = Lat_Lon_DF1.Longitude[0]
        Csv_DF["Latitude"] = Lat_Lon_DF1.Latitude[0]
        Csv_DF2 = pd.read_csv(DataFile2, header=[2], parse_dates={
            'Date': [0, 1, 2, 3, 4]}, date_parser=dateparse)
        Lat_Lon_DF2 = pd.read_csv(DataFile2, nrows=1)
        Csv_DF2["Longitude"] = Lat_Lon_DF2.Longitude[0]
        Csv_DF2["Latitude"] = Lat_Lon_DF2.Latitude[0]
        BigDF = BigDF.append([Csv_DF, Csv_DF2])
        # print(BigDF)
    elif len(files) == 1:
        DataFile = files.pop()
        Csv_DF = pd.read_csv(DataFile, header=[2], parse_dates={
            'Date': [0, 1, 2, 3, 4]}, date_parser=dateparse)
        Lat_Lon_DF1 = pd.read_csv(DataFile, nrows=1)
        Csv_DF["Longitude"] = Lat_Lon_DF1.Longitude[0]
        Csv_DF["Latitude"] = Lat_Lon_DF1.Latitude[0]
        BigDF = BigDF.append([Csv_DF])
        # print(BigDF)

Parsed_Data = BigDF.groupby(
    [pd.Grouper(key="Date", freq='1D'), "Latitude", "Longitude"]).mean()
# Parsed_Data = Parsed_Data.round(2)
print(Parsed_Data)

end = time.time()
print("Elapsed time parsed data: ", (end - start))


In [15]:
for name in ["DHI", "GHI", "DNI", "Air Temperature"]:
    start = time.time()
    if not os.path.isdir(data_dir+"2020"+name):
        os.makedirs(data_dir+"2020"+name)
    else:
        continue
    end = time.time()
    print("Elapsed time making directory: ", (end - start))


Elapsed time making directory:  0.0009908676147460938
Elapsed time making directory:  0.000990152359008789


In [20]:
expected_date = datetime.date(int(2020), 1, 1)

day_offset = datetime.timedelta(days=1)
for Value in ["DHI", "GHI", "DNI", "Air Temperature"]:
    start = time.time()
    Data = pd.DataFrame(columns=['value', 'latitude', 'longitude'])
    for rowIndex, row in Parsed_Data.iterrows():
        # print(f"rowIndex[0] = {rowIndex[0]}")
        # print(f"rowIndex[1] = {rowIndex[1]}")
        # print(f"rowIndex[2] = {rowIndex[2]}")
        # print(f"row = {row}")
        if rowIndex[0].date() == expected_date:
            Data = Data.append({'value': row[Value], 'latitude':
                                rowIndex[1], 'longitude': rowIndex[2]}, ignore_index=True)
            print(Data)
        elif expected_date.year > rowIndex[0].date().year and expected_date.day >= 2:
            expected_date = datetime.date(int(Year), 1, 1)
            # print(f"Expected date: {expected_date}")
            continue
        else:
            Data.to_csv(
                path_or_buf=f"{data_dir}{rowIndex[0].strftime('%Y')}{Value}/{Value}{expected_date.strftime('%Y%j')}.csv",
                header=False, index=False, sep=' ')
            Data.drop(index=Data.index, inplace=True)
            expected_date = expected_date+day_offset
            # print(f"This is the expected date after the offset: {expected_date}")
    print(Data)
end = time.time()
print("Elapsed time file generator: ", (end - start))


NameError: name 'Parsed_Data' is not defined